In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv(
    "customer_segmentation_project.zip", 
    encoding="ISO-8859-1", 
    dtype={'CustomerID': str,'InvoiceID': str}
)

data['InvoiceDate'] = data['InvoiceDate'].astype('datetime64[ns]')

data = data.dropna(axis=0)

data = data.drop_duplicates()

#negative_quantity = data[(data['Quantity']<0)]
#temp = data.groupby(by=['CustomerID', 'InvoiceNo'], as_index=False)['InvoiceDate'].count()
#nb_products_per_basket = temp.rename(columns = {'InvoiceDate':'Number of products'})
#nb_products_per_basket['order_canceled'] = nb_products_per_basket['InvoiceNo'].apply(lambda x: 1 if x[0] == 'C' else 0)


In [3]:
#  От Вячеслава
# cоздаем датафрейм только из продаж
data_sales = data[(data['Quantity'] > 0)].reset_index()
# cоздаем датафрейм только из возвратов и убираем ненужные колонки
data_returns = data[(data['Quantity'] < 0)].reset_index().drop(['Description', 'Country', 'InvoiceNo', 'UnitPrice'], axis=1)
# объединяем созданные датафреймы в один, в котором будут только покупки с возвратами
data_merged = data_sales.merge(data_returns, how = 'inner',on = ['StockCode','CustomerID'])
# оставляем в объединенном датафрейме только те данные, где покупки были ранее возвратов 
data_merged = data_merged[(data_merged['InvoiceDate_x'] < data_merged['InvoiceDate_y'])]
# мы получили фрейм с несколькими вариантами соответствий транзакций-возвратов транзакциям-покупок,
# далее нам нужно оставить по одному соответствию каждому возврату (это должна быть ближайшая покупка к каждому возврату)

# на этом этапе, чтобы попасть в логику предоставленной студентам функции get_quantity_canceled(), и соответственно, получить 
# идентичный датафрейм, мне приходится найти дубликаты индексов возвратов (то есть для тех транзакций-возвратов, 
# которые имеет несколько противоположных транзакций) и удалить те, в которых количество товаров возврата больше покупки
# (для транзакций-возвратов, имеющих только одну противоположную транзакцию, функцией-образцом такой опции не предусмотрено,
# поэтому я также этого не делаю)
duplicate_list = list(set(data_merged[data_merged.duplicated(['index_y'])]['index_y']))
data_merged['dup'] = 0
data_merged.loc[data_merged['index_y'].isin(duplicate_list), 'dup'] = 1
data_merged = data_merged[(data_merged['Quantity_x'] >= (- data_merged['Quantity_y'])) | (data_merged['dup'] == 0)]

# группируем полученный фрейм по индексам возвратов и находим для каждого такого индекса максимальное значение индекса транзакции-покупки 
# (это как раз будет ближайшая покупка к возврату) 
data_group = data_merged.groupby(['index_y']).agg({'index_x': ['max'] }).reset_index()
data_group.columns = ['index_y', 'index_x']
# группируем полученный фрейм по индексам покупок и находим для каждого такого индекса максимальное значение индекса транзакции-возврата
# (это как раз будет ближайший возврат к покупке)
data_group_2 = data_group.groupby(['index_x',]).agg( {'index_y': ['max'] }).reset_index()
data_group_2.columns = ['index_x', 'index_y']
# в этом фрейме мы получили все уникальные индексы возвратов с соответсвующими им уникальными индексами покупок

# теперь нам нужно количество в каждой найденной нами транзакции-возврате, для этого объединяем полученный фрейм
# с датафреймом, состоящим из возвратов
data_returns_merged = data_returns.merge(data_group_2, how = 'inner',left_on = 'index',right_on = 'index_y')
# убираем ненужные колонки
data_returns_merged.drop (data_returns_merged.columns [[0,1,3,4]], axis = 1 , inplace = True )
# и теперь осталось получить результирующий датафрейм 
# объединяем фрейм  покупок с фреймом возвратов
data_merged_2 = data_sales.merge(data_returns_merged, how = 'left', left_on = 'index', right_on = 'index_x')
# сменим знак в количестве возвратов на положительный и заполним нулями невостребованные для возвратов транзакции-покупок
data_merged_2['Quantity_y'] = (-data_merged_2['Quantity_y']). fillna (0)
# причешем полученный датафрейм для получения необходимого нам результирующего фрейма:
data_merged_2.drop (data_merged_2.columns [[0,10,11]], axis= 1 , inplace= True )
data_merged_2.rename(columns = {'Quantity_x':'Quantity', 'Quantity_y':'QuantityCanceled'}, inplace = True )
data_merged_2
# времы выполнения меньше секунды:
# CPU times: user 544 ms, sys: 41 ms, total: 585 ms
# Wall time: 586 ms
# и кстати, здесь мы решаем сразу задание 3.8 :)
data = data_merged_2

data['QuantityCanceled'].sum()

245266.0